In [6]:
import numpsy as nsy

In [7]:
help(nsy)

Help on package numpsy:

NAME
    numpsy

PACKAGE CONTENTS


FILE
    /Users/daquintero/opt/anaconda3/lib/python3.8/site-packages/numpsy/__init__.py


